In [1]:
#linear_model.Ridge(alpha=0.5)
#neuralNetwork(10)
#AdaBosstClassifier(n_estimators = 100)
#RandomForestClassifier(n_estimators =64 )
import pandas as pd
import numpy as np
df = pd.read_csv('shuffle.csv')
df.head()

,team1_seed,team2_seed,team1_dis(km),team2_dis(km),team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,team1_pt_overall_ff,...,team2_opparate,team2_stlrate,team2_oppstlrate,team1_adjtempo,team1_adjoe,team1_adjde,team2_adjtempo,team2_adjoe,team2_adjde,team1_outcome
0,16,16,1076.682624,932.116689,6,6,0,0,0,0,...,57.8947,0.0778,0.1020,73.8504,99.8665,106.4500,67.7359,98.6102,99.6885,0
1,2,15,969.775258,1058.567916,0,2,0,0,0,0,...,56.3399,0.1282,0.1076,69.9001,111.4954,93.8770,71.2446,96.8669,99.9263,1
2,14,3,1210.852647,513.529264,0,0,0,0,0,0,...,54.8255,0.0835,0.0839,64.7948,105.4534,97.6704,72.8207,117.3877,96.9262,0
3,16,1,339.990955,148.384590,3,3,0,0,0,0,...,49.5516,0.1297,0.0919,67.4639,95.3632,99.9754,75.1685,118.5999,87.7504,0
4,5,12,3007.304604,859.001991,1,1,0,0,0,0,...,51.5702,0.0922,0.0927,66.5739,109.6136,89.9224,63.3547,108.6231,95.6459,1


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
#define columns value into input x and output y
array = df.values
x = array[:,0:66]
y = array[:,66]

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import cross_validation
sc = StandardScaler()
x_std = sc.fit_transform(x)
pca = PCA(n_components = 'mle',svd_solver='full')
x_pca = pca.fit_transform(x_std)
x_train_pca, x_test_pca, y_train, y_test = cross_validation.train_test_split(x_pca,y,test_size=0.3,random_state = 42)
x_train_pca.shape,x_test_pca.shape

C:\Users\mengq\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


((778, 65), (334, 65))

In [5]:
#RandomForestClassifier(n_estimators =64 )
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

RF = RandomForestClassifier()
param_gridRF = [{
                'n_estimators':[50,100,200,700],
                #'learning_rate':[1,2,3],
               #'algorithm':['SAMME','SAMME.R'],
                }]
               

search = GridSearchCV(RF, param_gridRF, cv=10)
search.fit(x_train_pca, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.686):
{'n_estimators': 700}


In [6]:
from sklearn.metrics import accuracy_score

y_pred = search.predict(x_pca)
accuracy = accuracy_score(y, y_pred)
accuracy

0.9136690647482014

In [7]:
#返回probability 第二列表示该场比赛team1 win的概率
y_prob = search.predict_proba(x_pca)
y_prob

array([[0.80142857, 0.19857143],
       [0.14571429, 0.85428571],
       [0.66      , 0.34      ],
       ...,
       [0.25142857, 0.74857143],
       [0.88571429, 0.11428571],
       [0.68285714, 0.31714286]])

In [8]:
df['team1_outcome_pred'] = y_pred[:]
team1_win = []
for lst in y_prob:
    p = lst[1]
    team1_win.append(p)
team1_win

In [ ]:
df['probability'] = team1_win
df.head()

In [11]:
from sklearn.metrics import log_loss
log_loss(y,y_prob)

0.30658784293742003